In [7]:
from aocd import get_data
get_data(day=9)

'468 players; last marble is worth 71843 points'

# --- Day 9: Marble Mania ---
You talk to the Elves while you wait for your navigation system to initialize. To pass the time, they introduce you to their favorite marble game.

The Elves play this game by taking turns arranging the marbles in a circle according to very particular rules. The marbles are numbered starting with 0 and increasing by 1 until every marble has a number.

First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.

Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.

However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.

For example, suppose there are 9 players. After the marble with value 0 is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:
```
[-] (0)
[1]  0 (1)
[2]  0 (2) 1 
[3]  0  2  1 (3)
[4]  0 (4) 2  1  3 
[5]  0  4  2 (5) 1  3 
[6]  0  4  2  5  1 (6) 3 
[7]  0  4  2  5  1  6  3 (7)
[8]  0 (8) 4  2  5  1  6  3  7 
[9]  0  8  4 (9) 2  5  1  6  3  7 
[1]  0  8  4  9  2(10) 5  1  6  3  7 
[2]  0  8  4  9  2 10  5(11) 1  6  3  7 
[3]  0  8  4  9  2 10  5 11  1(12) 6  3  7 
[4]  0  8  4  9  2 10  5 11  1 12  6(13) 3  7 
[5]  0  8  4  9  2 10  5 11  1 12  6 13  3(14) 7 
[6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7(15)
[7]  0(16) 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[8]  0 16  8(17) 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[9]  0 16  8 17  4(18) 9  2 10  5 11  1 12  6 13  3 14  7 15 
[1]  0 16  8 17  4 18  9(19) 2 10  5 11  1 12  6 13  3 14  7 15 
[2]  0 16  8 17  4 18  9 19  2(20)10  5 11  1 12  6 13  3 14  7 15 
[3]  0 16  8 17  4 18  9 19  2 20 10(21) 5 11  1 12  6 13  3 14  7 15 
[4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
[5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
```
The goal is to be the player with the highest score after the last marble is used up. Assuming the example above ends after the marble numbered 25, the winning score is 23+9=32 (because player 5 kept marble 23 and removed marble 9, while no other player got any points in this very short example game).

Here are a few more examples:

- 10 players; last marble is worth 1618 points: high score is 8317
- 13 players; last marble is worth 7999 points: high score is 146373
- 17 players; last marble is worth 1104 points: high score is 2764
- 21 players; last marble is worth 6111 points: high score is 54718
- 30 players; last marble is worth 5807 points: high score is 37305

What is the winning Elf's score?

In [2]:
def place_marble(board, value, current_position):
    next_position = (current_position + 2) % (len(board))
    if next_position == 0: next_position = len(board)
    
    return (board[:next_position] + [value] + board[next_position:], next_position, 0)

def take_23(board, value, current_position):
    from_beginning = min(current_position, 7)
    removed_marble_index = (current_position - 7) % len(board)
    
    points = value + board[removed_marble_index]
    return (board[:removed_marble_index] + board[removed_marble_index + 1:], removed_marble_index, points)
        

def generate_moves():
    board, current_position, value = [0], 0, 0
    while True:
        value += 1
        action = take_23 if value % 23 == 0 else place_marble
        board, current_position, points = action(board, value, current_position)
        yield points

In [3]:
def player_points(players, max_value):
    points = defaultdict(int)
    for move, point_gain in zip(range(1, max_value + 1), generate_moves()):
        points[move % players] += point_gain
    return max(points.values())

In [4]:
player_points(468, 71843)

385820

# --- Part Two ---
Amused by the speed of your answer, the Elves are curious:

What would the new winning Elf's score be if the number of the last marble were 100 times larger?

In [5]:
class LinkedListNode:
    def __init__(self, value):
        self.value = value
        self.next_node = None
        self.previous_node = None
        
    def pop(self):
        self.previous_node.next_node = self.next_node
        self.next_node.previous_node = self.previous_node
        return self.next_node
        
    def append(self, new_node):
        new_node.previous_node = self
        new_node.next_node = self.next_node
        self.next_node.previous_node = new_node
        self.next_node = new_node
        return new_node
    
    def backwards(self, times):
        node = self
        for _ in range(times): 
            node = node.previous_node
        return node
    
    def as_list(self):
        assert self.value == 0
        result, node = [self.value], self.next_node
        while node.value != 0:
            result.append(node.value)
            node = node.next_node
        return result
        
    @staticmethod
    def start_node():
        node = LinkedListNode(0)
        node.next_node = node
        node.previous_node = node
        return node
    
    def __repr__(self):
        return "LinkedListNode({})".format(self.value)
        
def place_marble(node, value):
    return (node.next_node.append(LinkedListNode(value)), 0)

def take_23(node, value):
    removed_node = node.backwards(7)
    return (removed_node.pop(), removed_node.value + value)

def generate_moves():
    node, value = LinkedListNode.start_node(), 0
    
    while True:
        value += 1
        action = take_23 if value % 23 == 0 else place_marble
        node, points = action(node, value)
        yield points
        

In [6]:
def player_points(players, max_value):
    points = defaultdict(int)
    for move, point_gain in zip(range(1, max_value + 1), generate_moves()):
        points[move % players] += point_gain
    return max(points.values())

player_points(468, 7184300)

3156297594